# Apriori算法
--- 
## 关联分析Association Analysis

关联分析是一种在大规模数据集中寻找有趣关系的任务。 这些关系可以有两种形式:

- 频繁项集（frequent item sets）: 经常出现在一块的物品的集合。
- 关联规则（associational rules）: 暗示两种物品之间可能存在很强的关系。

支持度与可信度
- 支持度(support): 数据集中包含该项集(子集合)的记录所占的比例
- 可信度或置信度(confidence): 一条规则 A -> B 的可信度定义为 support(A | B) / support(A)。

`支持度`和 `可信度` 是用来量化` 关联分析 `是否成功的一个方法. 。 假设想找到支持度大于 0.8 的所有项集，应该如何去做呢？ 一个办法是生成一个物品所有可能组合的清单，然后对每一种组合统计它出现的频繁程度，但是当物品成千上万时，上述做法就非常非常慢了.

## Apriori算法原理
假设有4个商品{0, 1, 2, 3}的所有可能的项集合 2^4 - 1 = 15个.随着物品的增加，计算的次数呈指数的形式增长.

为了降低计算次数和时间，研究人员发现了一种所谓的 `Apriori `原理，即某个项集是频繁的，那么它的所有子集也是频繁的。 例如，如果 {0, 1} 是频繁的，那么 {0}, {1} 也是频繁的。 该原理直观上没有什么帮助，但是如果反过来看就有用了，也就是说如果一个项集是 `非频繁项集`，那么它的所有`超集`也是非频繁项集.
![Apriori](../img/Apriori.png)

In [50]:
from itertools import combinations
from functools import reduce

In [51]:
a = {0, 1, 2, 3}
union = []
for i in range(1, 5):
    union.extend(list(combinations(a, i)))

In [52]:
union

[(0,),
 (1,),
 (2,),
 (3,),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, 2),
 (1, 3),
 (2, 3),
 (0, 1, 2),
 (0, 1, 3),
 (0, 2, 3),
 (1, 2, 3),
 (0, 1, 2, 3)]

Apriori 算法优缺点

* 优点：易编码实现
* 缺点：在大数据集上可能较慢
* 适用数据类型：数值型 或者 标称型数据。

In [54]:
def create_c1(data_set):
    # 生成含单个元素的集合的列表
    C1 = []
    for transaction in data_set:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    # frozenset 不可变集合
    return list(map(set, C1))

In [55]:
data = [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]
c1 = create_c1(data)

In [56]:
reduce(set.union, c1)

{1, 2, 3, 4, 5}

In [79]:
class Apriori:
    def __init__(self, min_support=0.5):
        # 支持度 >= 最小支持度的候选项集以及它们的支持度。即我们的频繁项集。
        self.min_support = min_support
    @staticmethod
    def create_c1(data_set):
        # 生成含单个元素的集合的列表
        C_1 = []
        for transaction in data_set:
            for item in transaction:
                if not [item] in C_1:
                    C_1.append([item])
        C_1.sort()
        # frozenset 不可变集合  hashable
        return list(map(frozenset, C_1))
    
    @staticmethod
    def apriori_gen(ck, k):
        # 输入频繁项集列表 Lk 与返回的元素个数 k，然后输出所有可能的候选项集 Ck
        # lk = [{1, 2}, {0, 1}, {2, 3}] k=3 -> [{0, 1,2 }, {1, 2, 3}, {0, 2, 3}]
        union_set = reduce(set.union, map(set, ck))
        return list(map(frozenset, combinations(union_set, k)))
    
    def fit(self, data):
        c_k = self.create_c1(data)
        D = list(map(set, data))
        support_dict = self.scan(D, c_k)
        k = 2
        total_num = len(c_k)
        while k < total_num:
            # 由满足支持度条件的ck 生成的含2, 3 ..个元素组成的集合列表
            c_k = self.apriori_gen(c_k, k)
            dict_ = self.scan(D, c_k)
            if not dict_:
                break
            support_dict.update(dict_)
            k += 1
        return support_dict
            
    def scan(self, D, C_k):
        # 数据集ck在数据集D中的支持度, 
        # 并返回支持度大于最小支持度（minSupport）的数据
        
        # 计算频数
        cnt = {}
        for set_ in D:
            for C in C_k:
                # 若C为D中数据集的子集, C频数+1
                if C.issubset(set_):
                    cnt[C] = cnt.get(C, 0) + 1
        num = len(D)
        ret_C = {}
        for key in cnt:
            support = cnt[key] / num
            if support >= self.min_support:
                ret_C.update({key: support})
        return ret_C

In [80]:
apriori = Apriori(0.5)
apriori.fit(data)

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75,
 frozenset({1, 3}): 0.5,
 frozenset({2, 3}): 0.5,
 frozenset({2, 5}): 0.75,
 frozenset({3, 5}): 0.5,
 frozenset({2, 3, 5}): 0.5}

## 从频繁项集中挖掘关联规则
如下图所示，给出的是项集 {0,1,2,3} 产生的所有关联规则:
![2](../img/Apriori2.jpg)

通过观察，我们可以知道，如果某条规则并不满足 `最小可信度` 要求，那么该规则的所有子集也不会满足 最小可信度 的要求。
如上图所示，假设` 123 -> 3 `并不满足最小可信度要求，那么就知道任何左部为{0,1,2} 子集的规则也不会满足 最小可信度 的要求。 即` 12 -> 03 , 02 -> 13 , 01 -> 23 , 2 -> 013, 1 -> 023, 0 -> 123 `都不满足 最小可信度 要求。

In [ ]:
class ApriorRules(Apriori):
    def __init__(self, min_support=0.5, min_confidence=0.5)
        super().__init__(min_support)
        self.min_confidence = min_confidence
    
    def calc_confidence(self, set1, set2):
        # 规则 set1 -> set2 的置信度
        
        
    
    def fit(self, data):
        self.support_dict = super().fit(data)